In [3]:
import os
from time import time
from tqdm import tqdm
import numpy

from keras.preprocessing.image import ImageDataGenerator
import torch
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.transforms import transforms

from data import ChristmasImages
from model import Network


# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
full_dataset = ChristmasImages('image_data',True)
mymodel = Network()

D:\Users\jeeva\Python\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Users\jeeva\Python\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
!pip install natsort

In [2]:
# tfm = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(degrees=10),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

In [5]:
# TRAIN_ROOT = 'data/train'
# #TEST_ROOT = 'data/val'


# train_ds = ImageFolder(TRAIN_ROOT, transform=tfm)
#test_ds = ImageFolder(TEST_ROOT, transform=tfm)
val_size = int(0.2*len(full_dataset))
train_size = len(full_dataset)- val_size
train_data, val_data = random_split(full_dataset, [train_size, val_size])
# valloader = DataLoader(val_data, batch_size=64,shuffle=True)


In [6]:
LEN_TRAIN = len(full_dataset)

In [7]:
print(LEN_TRAIN)

3726


In [10]:
print(full_dataset[0])

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -

In [11]:
train_loader = DataLoader(full_dataset, batch_size = 64, shuffle=True)
valloader = DataLoader(val_data, batch_size=64,shuffle=True)

In [15]:
# model = resnet18(pretrained=True)
# # Replace Output of Fully Connected Layer with Number of Labels for our Classification Problem
# model.fc = Linear(in_features=512, out_features=8)
mymodel = mymodel.to(device)
# #model = model.cuda()    # only if your system supports Nvidia CUDA

In [ ]:
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [17]:
optimiser = Adam(mymodel.parameters(), lr=3e-4, weight_decay=0.0001)
loss_fn = CrossEntropyLoss()

In [18]:
for epoch in range(10):
    start = time()
    x = 0
    tr_acc = 0
    test_acc = 0
    
    # Train
    mymodel.train()
    
    with tqdm(train_loader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            optimiser.zero_grad()
            
            xtrain = xtrain.to(device)
            ytrain = ytrain.to(device)
            train_prob = mymodel(xtrain)
            train_prob = train_prob.cpu()
            
            loss = loss_fn(train_prob, ytrain)
            loss.backward()
            optimiser.step()
            
            # training ends
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / LEN_TRAIN
    
    # Evaluate
#     model.eval()
#     with torch.no_grad():
#         for xtest, ytest in test_loader:
#             xtest = xtest.to(device)
#             test_prob = model(xtest)
#             test_prob = test_prob.cpu()
            
#             test_pred = torch.max(test_prob,1).indices
#             test_acc += int(torch.sum(test_pred == ytest))
            
#         ep_test_acc = test_acc / LEN_TEST
    
    end = time()
    duration = (end - start) / 60
    
    print(f"Epoch: {epoch}, Time: {duration}, Loss: {loss}\nTrain_acc:{ep_tr_acc}")

100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [14:03<00:00, 14.30s/batch]


Epoch: 0, Time: 14.060747090975443, Loss: 0.7408455014228821
Train_acc:0.8011272141706924


100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [13:08<00:00, 13.37s/batch]


Epoch: 1, Time: 13.148314352830251, Loss: 0.26147639751434326
Train_acc:0.9168008588298443


100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [12:54<00:00, 13.12s/batch]


...

Epoch: 7, Time: 12.899110500017802, Loss: 0.008224870078265667
Train_acc:0.9876543209876543


100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [12:44<00:00, 12.96s/batch]


Epoch: 8, Time: 12.747182428836823, Loss: 0.016234751790761948
Train_acc:0.9868491680085884


100%|███████████████████████████████████████████████████████████████████████████████| 59/59 [12:56<00:00, 13.16s/batch]

Epoch: 9, Time: 12.939885771274566, Loss: 0.32240626215934753
Train_acc:0.9871175523349437


In [19]:
val_acc = 0
for images, labels in valloader:
    pred_val_labels = mymodel(images)
    val_acc += (pred_val_labels.max(dim=1)[1] == labels).sum().item()
print('val accuracy ', val_acc/len(val_data))

val accuracy  0.9798657718120806


In [ ]:
# Model Performance
sample_list= 'data\\val\\1.png'
from PIL import Image
import numpy as np

for image_path in sample_list:
    img = Image.open(sample_list)
    display(img.resize((224,224)))
    img_tensor = tfm(img)
    img_tensor = img_tensor[np.newaxis, :]
    img_tensor = img_tensor.to(device)
    pred_prob = model(img_tensor)
    pred = torch.max(pred_prob,1).indices
    pred = pred.item()
    print("========================================================")
# mg_tensor = transform(img)
# print(mg_tensor)   

In [ ]:
val_dir = 'data\\val'

def mean_std(x):
    dataset_mean_std = ImageFolder(x,transform = tfm)
    
    valloader_mean_std = DataLoader(train_ds_mean_std, batch_size=len(train_ds_mean_std), shuffle=True, num_workers = 1)
    
    images,labels = next(iter(valloader_mean_std ))
    
    mean,std = images.mean([0,2,3]), images.std([0,2,3])
    
    return mean,std

#load the train and test data

mean,std = mean_std(val_dir)

valset = ImageFolder(val_dir,transform = tfm )

In [20]:
mymodel.save_model()